<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Задача**

Обучить модель классифицировать комментарии на позитивные и негативные. Значение метрики качества *F1* на обученной модели не меньше 0.75.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Install

In [ ]:
!pip -q install transformers pandarallel fast_ml

### Import, Settings

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from tqdm import notebook, tqdm
from pandarallel import pandarallel

import torch
import transformers
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    TextClassificationPipeline,
    BertModel
)

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from fast_ml.model_development import train_valid_test_split

In [ ]:
# Переменные окружения
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:1024'
pandarallel.initialize(progress_bar=True, nb_workers=16)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [ ]:
# Константы для активации CUDA
use_gpu = torch.cuda.is_available()
device = torch.device("cuda")# if use_gpu else "cpu")

# Константы Bert
model_path = "JungleLee/bert-toxic-comment-classification"
tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2).to('cuda:0')
model = BertModel.from_pretrained(model_path).to('cuda:0')

### Загрузка датасета

Загрузим датасет

In [ ]:
df = pd.read_csv(
    'https://code.s3.yandex.net/datasets/toxic_comments.csv',sep=',', error_bad_lines=False
    )

Проверим наличие пропусков, изучим типы данных

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
# Блок использовался для ускорения отработки токенизации и эмбеддинга
df_subsample = df.sample(n=16000, replace=True, random_state=42).reset_index(drop=True)

Проверим наличие явных дубликатов

In [ ]:
df['text'].duplicated().sum()

0

Явных дубликатов нет.

Проверим баланс классов.

In [ ]:
df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]

8.841344371679229

Отрицательных значений в 8.8 раз больше, чем положительных. Учтем это в дальнейшем при обучении моделей.

### Токенизация

Проведем токенизацию при помощи BertTokenizer по заранее обученной модели. Для ускорения используем метод parallel_apply.

In [ ]:
tqdm.pandas()

In [ ]:
tokenized = df_subsample['text'].parallel_apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

### Пэддинг

Уравняем длины векторов по максимальному вектору. Недостающие элементы заполним нулями.

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [ ]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

Введем маску.

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)

### Эмбеддинг

Проведем эмбеддинг. Модель - BertModel. Вычисления произведем на GPU с использованием CUDA для ускорения процесса.

In [ ]:
batch_size = 200
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        batch = batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        

  0%|          | 0/80 [00:00<?, ?it/s]

Сохраним pickle файл для последующей загрузки при необходимости.

In [ ]:
with open ('embeddings.pickle', 'wb') as f:
    pickle.dump(embeddings, f)

In [ ]:
#with open('embeddings.pickle', 'rb') as f:
#    embeddings = pickle.load(f)

## Обучение

Подготовим данные для обучения.

In [ ]:
features = pd.DataFrame(np.concatenate(embeddings))
df_new = pd.concat([features, df_subsample['toxic']], axis=1)

Проверим размерность.

In [ ]:
# Удалим возникшие пустые строки
df_new.dropna(axis=0, inplace=True)

In [ ]:
df_new.shape

(16000, 769)

Разделим данные на тренировочные, валидационные и тестовые выборки.

In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(
    df_new, target='toxic',
    train_size=0.7, valid_size=0.2,
    test_size=0.1, random_state=42
    )

In [ ]:
X_train.shape, y_train.shape,  X_valid.shape, y_valid.shape,  X_test.shape, y_test.shape

((11200, 768), (11200,), (3200, 768), (3200,), (1600, 768), (1600,))

In [64]:
model_svc = SVC(C=5, class_weight='balanced')
model_svc.fit(X_train, y_train)
pred_svc = model_svc.predict(X_valid)
scoring_svc = f1_score(y_valid, pred_svc)
print('F1-score модели SVC:', scoring_svc)

F1-score модели SVC: 0.8776371308016877


In [65]:
pred_svc_test = model_svc.predict(X_test)
scoring_svc_test = f1_score(y_test, pred_svc_test)
print('F1-score модели SVC на тестовых данных:', scoring_svc_test)

F1-score модели SVC на тестовых данных: 0.8845208845208845


## Выводы

В результате выполнения задания отработан принцип работы с BERT - токенизация, пэддинг и эмбеддинг. Применены ряд методов ускорения работы с текстом - использование GPU, CUDA, pandarallel. Результат метрики F1-score на модели логистической регрессии составил {{0,88}}, что соотвествует условию задачи (не менее 0,75)